In [ ]:
maxlen = 128 # change subnyq sampling rate HERE.

def gendata(fp, nsamples):
    global snrs, mods, train_idx, test_idx, lbl
    with open(fp, 'rb') as f:
      Xd = pickle.load(f, encoding='latin1')
    
    snrs,mods = map(lambda j: sorted(list(set(map(lambda x: x[j], Xd.keys())))), [1,0])
    X = []  
    lbl = []
    for mod in mods:
      for snr in snrs:
        X.append(Xd[(mod, snr)])
        for i in range(Xd[(mod, snr)].shape[0]):
          lbl.append((mod, snr))
    X = np.vstack(X)
    
    print('Length of lbl', len(lbl))
    print('shape of X', X.shape)

    np.random.seed(2016)
    n_example = X.shape[0]
    n_train = n_example * 0.5
    train_idx = np.random.choice(range(0,n_example), size=int(n_train), replace=False)
    test_idx = list(set(range(0, n_example)) - set(train_idx))
    X_train = X[train_idx]
    X_test =  X[test_idx]
    
    def to_onehot(vec):
      vec_hot = np.zeros([len(vec), max(vec) + 1])
      vec_hot[np.arange(len(vec)), vec] = 1
      return vec_hot
    
    Y_train = to_onehot(list(map(lambda x: mods.index(lbl[x][0]), train_idx)))
    Y_test = to_onehot(list(map(lambda x: mods.index(lbl[x][0]), test_idx)))

    return (X_train,X_test,Y_train,Y_test)

def norm_pad_zeros(X_train,nsamples):
    print ("Pad:", X_train.shape)
    for i in range(X_train.shape[0]):
        X_train[i,:,0] = X_train[i,:,0]/la.norm(X_train[i,:,0],2)
    return X_train


def to_amp_phase(X_train,X_test,nsamples):
    X_train_cmplx = X_train[:,0,:] + 1j* X_train[:,1,:]
    X_test_cmplx = X_test[:,0,:] + 1j* X_test[:,1,:]
    
    X_train_amp = np.abs(X_train_cmplx)
    X_train_ang = np.arctan2(X_train[:,1,:],X_train[:,0,:])/np.pi
  

    X_train_amp = np.reshape(X_train_amp,(-1,1,nsamples))
    X_train_ang = np.reshape(X_train_ang,(-1,1,nsamples))
   
    X_train = np.concatenate((X_train_amp,X_train_ang), axis=1) 
    X_train = np.transpose(np.array(X_train),(0,2,1))
    
    X_test_amp = np.abs(X_test_cmplx)
    X_test_ang = np.arctan2(X_test[:,1,:],X_test[:,0,:])/np.pi
    
    
    X_test_amp = np.reshape(X_test_amp,(-1,1,nsamples))
    X_test_ang = np.reshape(X_test_ang,(-1,1,nsamples))
    
    X_test = np.concatenate((X_test_amp,X_test_ang), axis=1) 
    X_test = np.transpose(np.array(X_test),(0,2,1))

    X_train = np.transpose(X_train,axes=(0,2,1))
    X_test = np.transpose(X_test,axes=(0,2,1))
    return (X_train, X_test)

In [ ]:
xtrain1,xtest1,ytrain1,ytest1= gendata("/content/drive/MyDrive/RML2016.10a_dict.pickle.pkl",maxlen)
print(xtrain1.shape)
print(ytrain1.shape)

In [ ]:
print('length of X before to_amp_phase:', xtrain1.shape)
X_train_ap,X_test_ap = to_amp_phase(xtrain1,xtest1,maxlen)
print('length of X after to_amp_phase:', xtrain1.shape)
X_train_iq, Y_train_iq = xtrain1, ytrain1
X_test_iq, Y_test_iq = xtest1, ytest1
Y_train_ap, Y_test_ap = ytrain1, ytest1

In [ ]:
X_train_ap=np.reshape(X_train_ap,(-1,2,128,1))
X_test_ap=np.reshape(X_test_ap,(-1,2,128,1))